In [ ]:
import glob

#read one folder
files = glob.glob('BHSig260-Hindi\\1\\*.tif')

In [ ]:
files

In [ ]:
from itertools import combinations

combinations_list = list(combinations(files, 2))

# Print each combination
print(combinations_list)

In [ ]:
#one value of combination
x,y=combinations_list[0]

In [ ]:
label1=0 if "F" in  x.split("\\")[-1].split(".")[0] else 1
label2=0 if "F" in  y.split("\\")[-1].split(".")[0] else 1
label=label1*label2

In [ ]:
x,y

In [ ]:
label

# general function

In [ ]:
import pandas as pd
import glob
from itertools import combinations

def files_to_csv(file_name, n, file_type):
    image1 = []
    image2 = []
    label = []
    labels1=[]
    labels2=[]
    
    for i in range(1, n+1):
        files = glob.glob(f'{file_name}\\{i}\\*.{file_type}')
        pairs = list(combinations(files, 2))
        
        for combo in pairs:
            label1 = 0 if "F" in  combo[0].split("\\")[-1].split(".")[0]  else 1
            label2 = 0 if "F" in combo[1].split("\\")[-1].split(".")[0]  else 1
            label.append(label1 * label2)
            image1.append(combo[0])
            labels1.append(label1)
            labels2.append(label2)
            image2.append(combo[1])
    
    # Create the DataFrame
    df = pd.DataFrame(list(zip(image1, image2, labels1,labels2,label)), columns=['Testing_Signature', 'Original_Signature','Label1','Label2', 'Ouput'])
    #remove all the rows where label2 is 0
    df=df[df.Label2!=0]
    df.to_csv(f"{file_name}_extracted.csv",index=False)

In [ ]:
files_to_csv("BHSig260-Hindi",160,"tif")

In [ ]:
df=pd.read_csv("BHSig260-Hindi_extracted.csv")

In [ ]:
import pandas as pd
import glob
from itertools import combinations

def cedar_files_to_csv(n, file_type):
    image1 = []
    image2 = []
    label = []
    labels1=[]
    labels2=[]
    
    for i in range(1, n+1):
        files = glob.glob(f'CEDAR\\{i}\\*.{file_type}')
        pairs = list(combinations(files, 2))
        
        for combo in pairs:
            # print(combo[0].split("\\")[-1][0])
            label1 = 0 if "f" ==  combo[0].split("\\")[-1][0]  else 1
            label2 = 0 if "f" == combo[1].split("\\")[-1][0]  else 1
            labels1.append(label1)
            labels2.append(label2)
            label.append(label1 * label2)
            image1.append(combo[0])
            image2.append(combo[1])
    
    # Create the DataFrame
    df = pd.DataFrame(list(zip(image1, image2, labels1,labels2,label)),  columns=['Testing_Signature', 'Original_Signature','Label1','Label2', 'Ouput'])
    df=df[df.Label2!=0]
    df.to_csv(f"CEDAR_extracted.csv",index=False)

In [ ]:
cedar_files_to_csv(55,"png")

In [ ]:
df1=pd.read_csv("CEDAR_extracted.csv")

In [ ]:
df1.head()

In [ ]:
#remove all the rows where label2 is 0
df1=df1[df1.Label2!=0]

In [ ]:
df1.head()

In [ ]:
#train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df1[['Testing_Signature', 'Original_Signature']], df1['Ouput'], test_size=0.2, random_state=42,
                                                    shuffle=True, stratify=df1['Ouput'])

In [ ]:
train=pd.DataFrame(list(zip(X_train['Testing_Signature'],X_train['Original_Signature'],y_train)),  columns=['Testing_Signature', 'Original_Signature', 'Ouput'])
test=pd.DataFrame(list(zip(X_test['Testing_Signature'],X_test['Original_Signature'],y_test)),  columns=['Testing_Signature', 'Original_Signature', 'Ouput'])
train.to_csv("CEDAR_train.csv",index=False)
test.to_csv("CEDAR_test.csv",index=False)

In [ ]:
from torchvision import transforms
from model import CustomDataset
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()])

# Create an instance of the CustomDataset
dataset = CustomDataset('CEDAR_extracted.csv',transform=transform)


In [ ]:
from model import CustomDataModule
dm = CustomDataModule('CEDAR_train.csv',transform=transform)
dm.setup()

# Get the training data loader
train_loader = dm.train_dataloader()

# Iterate over the batches and print the data
for batch_idx, (data1, data2, label) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    print("Data 1:")
    print(data1.shape)
    print("Data 2:")
    print(data2.shape)
    print("Label:")
    print(label)
    break

In [ ]:
import lightning as l
import torch
from model import SiameseNetwork,LightningModel
torch.manual_seed(123)
dm = CustomDataModule('CEDAR_train.csv',batch_size=64,transform=transform)

pytorch_model = SiameseNetwork()

lightning_model = LightningModel(model=pytorch_model, learning_rate=0.05)

trainer = l.Trainer(
    max_epochs=15,
    accelerator="cpu",
    devices="auto",
    deterministic=True,
)

trainer.fit(model=lightning_model, datamodule=dm)

In [ ]:
pytorch_model